## Configs

In [7]:
import sys
import os 

GRADIENT = os.path.exists('train')
KAGGLE = os.path.exists('../input')

if KAGGLE:
    sys.path.insert(0, '../input/uwmadisonutils/')
    data_path = '../input/uw-madison-gi-tract-image-segmentation/'
    
elif GRADIENT:
    sys.path.insert(0, 'uwmadisonutils/')
    data_path = ''

from uwmadisonutils.utilities import *

In [8]:
if KAGGLE:
    !cp -r ../input/pytorch-segmentation-models-lib/ ./
    !pip install -q ./pytorch-segmentation-models-lib/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4
    !pip install -q ./pytorch-segmentation-models-lib/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3
    !pip install -q ./pytorch-segmentation-models-lib/timm-0.4.12-py3-none-any.whl
    !pip install -q ./pytorch-segmentation-models-lib/segmentation_models_pytorch-0.2.0-py3-none-any.whl

## Prepare Data

In [9]:
path = Path(data_path+'train')
test_path = Path(data_path+'test')

train = pd.read_csv(data_path+'train.csv', low_memory=False)
sample_submission = pd.read_csv(data_path+'sample_submission.csv', low_memory=False).rename(columns={'predicted': 'segmentation'})

train_path = Path(data_path+'train')
test_path = Path(data_path+'test')

train_fnames = get_image_files(train_path)
test_fnames = get_image_files(test_path)

## Train and Export

In [10]:
run = 'exp-35'

In [11]:
channels = 3
stride = 2

train = pd.read_csv(data_path+'train.csv', low_memory=False)
train = get_custom_df(train, train_fnames, str(train_path), channels=channels, stride=stride)

In [12]:
def get_train_aug(img_size, crop=0.9, resize=0.5, p=0.4):
    crop_size = round(img_size[0]*crop)
    resize = round(crop_size*resize)
    return albumentations.Compose([
                albumentations.RandomCrop(height=crop_size, width=crop_size, always_apply=True),
                albumentations.HorizontalFlip(),
                albumentations.OneOf([
                    albumentations.Sharpen(),
                    albumentations.Emboss(),
                    albumentations.RandomGamma(),
                    albumentations.RandomBrightnessContrast(),
                    ], p=p),
                albumentations.OneOf([
                    albumentations.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
                    albumentations.GridDistortion(),
                    albumentations.OpticalDistortion(distort_limit=2, shift_limit=0.5),
                    
                    ], p=p),
                albumentations.ShiftScaleRotate(),
                albumentations.CoarseDropout()
            ])

def get_test_aug(img_size, resize=0.5, crop=0.9):
    crop_size = [round(crop*size) for size in img_size]
    resize = [round(resize*size) for size in crop_size]
    
    if crop < 1.0:
        crop_size = [crop_size[0], crop_size[0]]
        resize = [resize[0], resize[0]]
    
    return  albumentations.Compose([
        albumentations.CenterCrop(height=crop_size[0], width=crop_size[1], always_apply=True),
        albumentations.Resize(height=resize[0], width=resize[1], always_apply=True)
    ])

In [13]:
def deep_supervision_loss(inp, targ, loss_func=ComboLoss()):
    loss = 0
    for i in range(len(inp)):
        loss += loss_func(inp[i], targ)
    return loss/len(inp)

In [14]:
tfms_kwargs = dict(train=get_train_aug, test=get_test_aug)

In [16]:
dls, dev = get_5C_25D_dls(train, channels=channels, bs=64, crop=0.6, val_crop=1.0, resize=1.0, aug='albumentations',
                          sample_empty=False, val='group', fold=fold, tfms_kwargs=tfms_kwargs,
                          show=False, aug_p=0.5)

/opt/conda/lib/python3.8/site-packages/torch/_tensor.py:1131: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


In [25]:
learn = get_custom_learner(dls,
                           UNetPlusPlus('regnety_160', channels, 3, pretrained=True),
                           deep_supervision_loss,
                           unetplusplus_splitter,
                           [dice_coeff_deep, custom_metric_deep],
                           cbs=[GradientAccumulation(64), MixUp()])

In [27]:
learn.model = learn.model.to('cuda:0')

In [15]:
for fold in range(5):
    wandb.init(project="uw-madison models", group=f'{run}', name=f'{run}-f{fold}')
    
    dls, dev = get_5C_25D_dls(train, channels=channels, bs=64, crop=0.6, val_crop=1.0, resize=1.0, aug='albumentations',
                          sample_empty=False, val='group', fold=fold, tfms_kwargs=tfms_kwargs,
                          show=False, aug_p=0.5)
    
    learn = get_custom_learner(dls,
                           UNetPlusPlus('regnety_160', channels, 3, pretrained=True),
                           deep_supervision_loss,
                           unetplusplus_splitter,
                           [dice_coeff_deep, custom_metric_deep],
                           cbs=[GradientAccumulation(64), MixUp()])
    
    learn.unfreeze()
    
    learn.fit_one_cycle(100, 6e-3, cbs=[SaveModelCallback('dice_coeff_deep', fname=f'{run}-f{fold}', with_opt=True, save_at_epoch=99),
                                        WandbCallback(log_preds=False, model_name=f'{run}-f{fold}')])
    
    learn.export(f'{run}-f{fold}.pkl')

NameError: name 'wandb' is not defined

## Export only

In [ ]:
for fold in range(5):
    dls, dev = get_5C_25D_dls(train, channels=channels, bs=64, crop=0.6, val_crop=1.0, resize=1.0, aug='albumentations',
                          sample_empty=False, val='group', fold=fold, tfms_kwargs=tfms_kwargs,
                          show=False, aug_p=0.5)
    
    learn = get_custom_learner(dls,
                           UNetPlusPlus('regnety_160', channels, 3, pretrained=True),
                           deep_supervision_loss,
                           unetplusplus_splitter,
                           [dice_coeff_deep, custom_metric_deep],
                           cbs=[GradientAccumulation(64), MixUp()])
    
    learn.unfreeze()
    
    learn.load(f'{run}-f{fold}')
    